In [ ]:
# Import common libraries
import numpy as np
import pandas as pd

# Import MNE processing
import mne
from mne.preprocessing.nirs import optical_density, beer_lambert_law
from mne.preprocessing.nirs import optical_density, beer_lambert_law, scalp_coupling_index, temporal_derivative_distribution_repair


# Import MNE-NIRS processing
from mne_nirs.statistics import run_glm
from mne_nirs.experimental_design import make_first_level_design_matrix
from mne_nirs.statistics import statsmodels_to_results
from mne_nirs.channels import get_short_channels, get_long_channels
from mne_nirs.channels import picks_pair_to_idx
from mne_nirs.visualisation import plot_glm_group_topo
from mne_nirs.datasets import fnirs_motor_group
from mne_nirs.visualisation import plot_glm_surface_projection
from mne_nirs.io.fold import fold_channel_specificity

# Import MNE-BIDS processing
from mne_bids import BIDSPath, read_raw_bids, get_entity_vals

# Import StatsModels
import statsmodels.formula.api as smf

# Import Plotting Library
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from itertools import compress
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from scipy import signal

# Import other randoms
import h5py
from scipy import interpolate

import ipyevents
import pyvistaqt
import ipympl

In [ ]:
root = "Define your own root based on where the data set is saved"

print(root)

In [ ]:
# Make sure your data is in BIDS format before running the script

dataset = BIDSPath(root= "Define your own root based on where the data set is saved"
, 
                   datatype="nirs", suffix="nirs", extension=".snirf")
print(dataset.directory)

In [ ]:
# create lowpass filter
from scipy.signal import butter, filtfilt
def lowpass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs  # Nyquist frequency
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data
    

In [ ]:
# lag
import numpy as np
from scipy import signal

# Find optimal lag between two signals based on cross-correlation
def find_optimal_lag(nirs_data, aux_data, max_lag_samples=30):
    nirs = nirs_data - np.mean(nirs_data)
    aux = aux_data - np.mean(aux_data)
    correlation = np.correlate(aux, nirs, mode='full')
    lags = signal.correlation_lags(len(aux), len(nirs), mode='full')

    # Limit lags to a specific window
    valid_idx = (lags >= 0) & (lags <= max_lag_samples)
    correlation = correlation[valid_idx]
    lags = lags[valid_idx]

    best_lag = lags[np.argmax(np.abs(correlation))]
    best_corr = correlation[np.argmax(np.abs(correlation))]

    return best_lag, best_corr

# Create a lagged version of a dataset
def create_lagged_matrix(data, lags):
    return np.hstack([np.roll(data, shift=lag, axis=0) for lag in lags])


In [ ]:
# Define the individual analysis function to be used for each individual dataset
# Set up necessary parameters for design matrix and GLM process-- this is specific for the SS Correction + tCCA denoising model

def individual_analysis(bids_path, ID):
    
    # Load in data; remove unnecessary annotations and rename meaningful annotations
    raw_intensity = mne.io.read_raw_snirf(fname=bids_path, verbose=False, optode_frame="mri")
    raw_intensity.annotations.delete(raw_intensity.annotations.description == '15')
    raw_intensity.annotations.rename({'1': 'speech', '2': 'speech', '3': 'silence'})
    
    # sanitize event names
    raw_intensity.annotations.description[:] = [d.replace('/', '_') for d in raw_intensity.annotations.description]

    # Convert signal to haemoglobin and resample; remove poorly coupled optodes
    raw_od = optical_density(raw_intensity)
    sci = scalp_coupling_index(raw_od, h_freq=1.35, h_trans_bandwidth=0.1)
    raw_od.info['bads'] = list(compress(raw_od.ch_names, sci < 0.7))
    raw_od.interpolate_bads()
    raw_od.resample(.6)
    raw_haemo = beer_lambert_law(raw_od, ppf=0.1)

    # IDing short chans
    sht_chans = get_short_channels(raw_haemo)
    raw_haemo = get_long_channels(raw_haemo)

    # Create a design matrix
    design_matrix = make_first_level_design_matrix(
        raw_haemo, drift_model='cosine', high_pass=0.015, hrf_model='spm', stim_dur=6.0)
    
    # Append short channels mean to design matrix
    design_matrix["ShortHbO"] = np.mean(sht_chans.copy().pick(picks="hbo").get_data(), axis=0)
    design_matrix["ShortHbR"] = np.mean(sht_chans.copy().pick(picks="hbr").get_data(), axis=0)

    
    physio_cols = []
    # loading in aux signals
    for n in [18, 19, 20, 21, 22, 23]:
        with h5py.File(bids_path, 'r') as dat:
            aux = np.array(dat.get(f'nirs/aux{n}/dataTimeSeries'))
            aux_time = np.array(dat.get(f'nirs/aux{n}/time'))
            fqs = np.argmax(aux_time > 1)
            filtered_aux = lowpass_filter(aux, cutoff=0.5, fs=fqs, order=4)
            aux_data_interp = interpolate.interp1d(aux_time, filtered_aux, axis=0, bounds_error=False, fill_value='extrapolate')
            aux_data_matched_to_fnirs = aux_data_interp(raw_haemo.times)
            signal_name = np.array(dat.get(f'nirs/aux{n}/name'))[0].decode()
            design_matrix[signal_name] = aux_data_matched_to_fnirs
            physio_cols.append(signal_name)

    physio_data = design_matrix[physio_cols].values
    physio_data = StandardScaler().fit_transform(physio_data)

    fs = 1 / np.median(np.diff(raw_haemo.times))
    max_lag_seconds = 10
    max_lag_samples = int(max_lag_seconds * fs)
    lags = np.arange(-max_lag_samples, max_lag_samples + 1)

    # below is where we identify each "best lag" for each aux signal
    for i, col in enumerate(physio_cols):
        best_lag, _ = find_optimal_lag(physio_data[:, 0], physio_data[:, i], max_lag_samples=max_lag_samples)        
        physio_data[:, i] = np.roll(physio_data[:, i], best_lag)

    physio_lagged = create_lagged_matrix(physio_data, lags)
    valid_idx = max(abs(lags))
    physio_lagged = physio_lagged[valid_idx:-valid_idx]

    fnirs_data = raw_haemo.copy().pick(picks=["hbo", "hbr"]).get_data().T
    fnirs_data = StandardScaler().fit_transform(fnirs_data)
    fnirs_lagged = fnirs_data[valid_idx:-valid_idx]
    

    #CCA component
    cca = CCA(n_components=2, tol=0.3)
    physio_c, fnirs_c = cca.fit_transform(physio_lagged, fnirs_lagged)

    # Matching lengths of nirs and physio data; making sure raw_haemo contains both and is the same length
    # Only best lags used
    start = valid_idx
    end = -valid_idx if valid_idx > 0 else None
    design_matrix = design_matrix.iloc[start:end].reset_index(drop=True)
    raw_haemo.crop(tmin=raw_haemo.times[start], tmax=raw_haemo.times[end - 1] if end is not None else raw_haemo.times[-1])

    # Pair physiology data to design matrix
    design_matrix["CCA1"] = physio_c[:, 0]
    design_matrix["CCA2"] = physio_c[:, 1]

    glm_est = run_glm(raw_haemo, design_matrix)

    # ID channels to group together for ROIs
    IFG = [[1,1],[2,1],[3,1],[4,1],[3,2],[4,2],[9,8],[10,8],[11,8],[12,8],[11,9],[12,9]]
    HG = [[5,4],[5,3],[5,5],[6,4],[6,3],[6,5],[13,11],[13,10],[13,12],[14,11],[14,10],[14,12]]
    PT = [[6,6],[6,7],[7,5],[7,7],[8,6],[8,7],[14,13],[14,14],[15,12],[15,14],[16,13],[16,14]]

    # Define ROI groups to pair NIRS data to based on channels
    groups = dict(IFG = picks_pair_to_idx(raw_haemo, IFG),
                  PT = picks_pair_to_idx(raw_haemo, PT),
                  HG = picks_pair_to_idx(raw_haemo, HG)) 
    
    # Extract channel metrics
    cha = glm_est.to_dataframe()

    # Compute region of interest results from channel data
    roi = glm_est.to_dataframe_region_of_interest(groups, design_matrix.columns, demographic_info=True)

    
    # Define contrasts for conditions
    contrast_matrix = np.eye(design_matrix.shape[1])
    basic_conts = dict([(column, contrast_matrix[i])
                        for i, column in enumerate(design_matrix.columns)])
    contrast_SpchSil = basic_conts['speech'] - basic_conts['silence']

    # Compute defined contrast
    contrast = glm_est.compute_contrast(contrast_SpchSil)
    con = contrast.to_dataframe()

    # Add the participant ID to the dataframes
    roi["ID"] = cha["ID"] = con["ID"] = ID

    # Convert to uM for nicer plotting below.
    cha["theta"] = [t * 1.e6 for t in cha["theta"]]
    roi["theta"] = [t * 1.e6 for t in roi["theta"]]
    con["effect"] = [t * 1.e6 for t in con["effect"]]

    return raw_haemo, roi, cha, con, design_matrix

# analysis

In [ ]:
df_roi = pd.DataFrame()  # To store region of interest results
df_cha = pd.DataFrame()  # To store channel level results
df_con = pd.DataFrame()  # To store channel level contrast results

# Create path to file based on experiment info
for sub in [1, 2, 3, 4 ,5, 7, 8, 9, 11, 12, 13, 14, 17, 18, 19]: # change range values to specify the number of subject recordings to be analyzed
    ID = '%02d' % sub
    bids_path = BIDSPath(subject="%02d" % sub,
                         session="02", # change based on session being analyzed 
                         datatype="nirs",
                         root="C:\\Users\\abbie\\Box\\BRAiN Lab\\current projects\\esp-project\\esp-repeatability\\source data new glm\\", # Change to where data set is located
                         extension= ".snirf")
    raw_haemo, roi, cha, con, design_matrix= individual_analysis(bids_path, ID)
    df_roi = pd.concat([df_roi, roi], ignore_index=True)
    df_cha = pd.concat([df_cha, cha], ignore_index=True)
    df_con = pd.concat([df_con, con], ignore_index=True)

In [ ]:
# Visualize individual results

grp_results = df_roi.query("Condition in ['speech', 'silence']")
grp_results = grp_results.query("Chroma in ['hbo']")

sns.catplot(x="Condition", y="theta", col="ID", hue="ROI", data=grp_results, col_wrap=5, errorbar=None, palette="muted", height=4, s=10)

In [ ]:
### GROUP-LEVEL RESULTS ###

grp_results = df_roi.query("Condition in ['speech', 'silence']")

roi_model = smf.mixedlm("theta ~ -1 + ROI:Condition:Chroma",
                        grp_results, groups=grp_results["ID"]).fit(method='nm')
roi_model.summary()

In [ ]:
# Output of channel and roi-level hemodynamic response amplitudes from analysis

df_roi.to_csv("SS-tCCA_correction_model_ROI.csv") # ROI-level
df_cha.to_csv("SS-tCCA_correction_model_Chan.csv") #Channel-level

In [ ]:
# Regenerate the results from the original group model above; visualize group-level results
grp_results = df_roi.query("Condition in ['speech','silence']")
roi_model = smf.mixedlm("theta ~ -1 + ROI:Condition:Chroma",
                        grp_results, groups=grp_results["ID"]).fit(method='nm')

df = statsmodels_to_results(roi_model)

sns.catplot(x="Condition", y="Coef.", hue="ROI", data=df.query("Chroma == 'hbo'"), errorbar=None, palette="muted", height=4, s=10)

In [ ]:
# Group topographic visualization 

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10),
                         gridspec_kw=dict(width_ratios=[1, 1]))

groups_single_chroma = dict(
    HG = picks_pair_to_idx (raw_haemo.copy().pick(picks='hbo'),[[5,4],[5,3],[5,5],[6,4],[6,3],[6,5],[13,11],[13,10],[13,12],[14,11],[14,10],[14,12]] , on_missing = 'warning'),
    PT = picks_pair_to_idx (raw_haemo.copy().pick(picks= 'hbo'),[[6,6],[6,7],[7,5],[7,7],[8,6],[8,7],[14,13],[14,14],[15,12],[15,14],[16,13],[16,14]], on_missing = 'warning'),
    IFG = picks_pair_to_idx (raw_haemo.copy().pick(picks = 'hbo'), [[1,1],[2,1],[3,1],[4,1],[3,2],[4,2],[9,8],[10,8],[11,8],[12,8],[11,9],[12,9]], on_missing = 'warning'))

    
    # Cut down the dataframe just to the conditions we are interested in

    
ch_summary = df_cha.query("Condition in ['speech', 'silence']")  #####Change these to your conditions####
ch_summary = ch_summary.query("Chroma in ['hbo']")

# Run group level model and convert to dataframe
ch_model = smf.mixedlm("theta ~ -1 + ch_name:Chroma:Condition",
                       ch_summary, groups=ch_summary["ID"]).fit(method='nm')
ch_model_df = statsmodels_to_results(ch_model)

# Plot the two conditions for each ROI with both hbo and hbr

# plot group topographic data for HbO
plot_glm_group_topo(raw_haemo.copy().pick(picks="hbo").pick(groups_single_chroma['IFG']),
                    ch_model_df.query("Condition in ['speech']"),                
                    colorbar=False, axes=axes[0, 0],
                    vlim=(0, 4), cmap=mpl.cm.Oranges)

plot_glm_group_topo(raw_haemo.copy().pick(picks="hbo").pick(groups_single_chroma['IFG']),
                    ch_model_df.query("Condition in ['silence']"),              
                    colorbar=True, axes=axes[0, 1],
                    vlim=(0, 4), cmap=mpl.cm.Oranges)

plot_glm_group_topo(raw_haemo.copy().pick(picks="hbo").pick(groups_single_chroma['HG']),
                    ch_model_df.query("Condition in ['speech']"),                
                    colorbar=False, axes=axes[0, 0],
                    vlim=(0, 4), cmap=mpl.cm.Oranges)

plot_glm_group_topo(raw_haemo.copy().pick(picks="hbo").pick(groups_single_chroma['HG']),
                    ch_model_df.query("Condition in ['silence']"),              
                    colorbar=True, axes=axes[0, 1],
                    vlim=(0, 4), cmap=mpl.cm.Oranges)

plot_glm_group_topo(raw_haemo.copy().pick(picks="hbo").pick(groups_single_chroma['PT']),
                    ch_model_df.query("Condition in ['speech']"),                
                    colorbar=False, axes=axes[0, 0],
                    vlim=(0, 4), cmap=mpl.cm.Oranges)

plot_glm_group_topo(raw_haemo.copy().pick(picks="hbo").pick(groups_single_chroma['PT']),
                    ch_model_df.query("Condition in ['silence']"),              
                    colorbar=True, axes=axes[0, 1],
                    vlim=(0, 4), cmap=mpl.cm.Oranges)


# deoxygenated hemoglobin (HbR)
ch_summary = df_cha.query("Condition in ['speech', 'silence']")         
ch_summary = ch_summary.query("Chroma in ['hbr']")

# Run group level model and convert to dataframe
ch_model = smf.mixedlm("theta ~ -1 + ch_name:Chroma:Condition",
                       ch_summary, groups=ch_summary["ID"]).fit(method='nm')
ch_model_df = statsmodels_to_results(ch_model)


# plot group topographic data for HbR
plot_glm_group_topo(raw_haemo.copy().pick(picks="hbr").pick(groups_single_chroma['IFG']),
                    ch_model_df.query("Condition in ['speech']"),                
                    colorbar=False, axes=axes[0, 0],
                    vlim=(0, 4), cmap=mpl.cm.Blues_r)

plot_glm_group_topo(raw_haemo.copy().pick(picks="hbr").pick(groups_single_chroma['IFG']),
                    ch_model_df.query("Condition in ['silence']"),               
                    colorbar=True, axes=axes[0, 1],
                    vlim=(0, 4), cmap=mpl.cm.Blues_r)

plot_glm_group_topo(raw_haemo.copy().pick(picks="hbr").pick(groups_single_chroma['HG']),
                    ch_model_df.query("Condition in ['speech']"),                
                    colorbar=False, axes=axes[1, 0],
                    vlim=(-10, 0), cmap=mpl.cm.Blues_r)
plot_glm_group_topo(raw_haemo.copy().pick(picks="hbr").pick(groups_single_chroma['HG']),
                    ch_model_df.query("Condition in ['silence']"),               
                    colorbar=True, axes=axes[1, 1],
                    vlim=(-10, 0), cmap=mpl.cm.Blues_r)

plot_glm_group_topo(raw_haemo.copy().pick(picks="hbr").pick(groups_single_chroma['PT']),
                    ch_model_df.query("Condition in ['speech']"),                
                    colorbar=False, axes=axes[1, 0],
                    vlim=(-10, 0), cmap=mpl.cm.Blues_r)
plot_glm_group_topo(raw_haemo.copy().pick(picks="hbr").pick(groups_single_chroma['PT']),
                    ch_model_df.query("Condition in ['silence']"),               
                    colorbar=True, axes=axes[1, 1],
                    vlim=(-10, 0), cmap=mpl.cm.Blues_r)

plt.show()

In [ ]:
# Contrast topographic visualization 

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
con_summary = df_con.query("Chroma in ['hbo']")

# Run group level model and convert to dataframe
con_model = smf.mixedlm("effect ~ -1 + ch_name:Chroma",
                        con_summary, groups=con_summary["ID"]).fit(method='nm')
con_model_df = statsmodels_to_results(con_model,
                                      order=raw_haemo.copy().pick(
                                          picks="hbo").ch_names)

#contrast plot for each ROI
plot_glm_group_topo(raw_haemo.copy().pick(picks="hbo").pick(groups_single_chroma['IFG']),
                    con_model_df, colorbar=True, vlim=(-5, 5), axes=axes)

plot_glm_group_topo(raw_haemo.copy().pick(picks="hbo").pick(groups_single_chroma['HG']),
                    con_model_df, colorbar=True, vlim=(-5, 5), axes=axes)

plot_glm_group_topo(raw_haemo.copy().pick(picks="hbo").pick(groups_single_chroma['PT']),
                    con_model_df, colorbar=True, vlim=(-5, 5), axes=axes)


In [ ]:
### Here, plot beta estimates on the cortical surface for specified conditions
# Generate brain figure from channel-level data

# Contrast model
subjects_dir = mne.datasets.sample.data_path() / 'subjects'
clim = dict(kind='value', pos_lims=(0, 8, 11))
brain = plot_glm_surface_projection(raw_haemo.copy().pick("hbo"),
                                    con_model_df, clim=clim, view='dorsal',
                                    colorbar=True, size=(800, 700), subjects_dir=subjects_dir)
brain.add_text(0.05, 0.95, "speech-silence", 'title', font_size=16, color='k')

# Run model code as above
clim = dict(kind='value', pos_lims=(0, 11.5, 17))

    ##########################################################
    #   Pick conditions and list them in the line below   # 
    ##########################################################

# Models for both conditions
for idx, cond in enumerate(['speech','silence']):

    # Run same model as explained in the sections above
    ch_summary = df_cha.query("Condition in [@cond]")
    ch_summary = ch_summary.query("Chroma in ['hbo']")
    ch_model = smf.mixedlm("theta ~ -1 + ch_name", ch_summary,
                           groups=ch_summary["ID"]).fit(method='nm')
    model_df = statsmodels_to_results(ch_model, order=raw_haemo.copy().pick("hbo").ch_names)

    # Generate brain figure from data
    brain = plot_glm_surface_projection(raw_haemo.copy().pick("hbo"),
                                        model_df, clim=clim, view='dorsal', value='Coef.',
                                        colorbar=True, size=(800, 700), subjects_dir=subjects_dir)
    brain.add_text(0.05, 0.95, cond, 'title', font_size=16, color='k')